In [1]:
import pickle
import glob
from tqdm import tqdm

# load the sequences and cast them
files = glob.glob("./proteins/*/sequences/sequences_pck.pkl")

number_of_protein = 14
def get_seq_class(files, mapping):
    # list with all the sequences
    sequences_class = []
    sequences_dict = dict()

    for file in files:
        # load the sequence
        with open(file, 'rb') as f:
            seq = pickle.load(f)

        s_seq = dict()
        # cast all the sequences as a string 
        for key in seq.keys():
            # I have to check and filter out the 
            # sequences with J otherwise the model
            # get an error
            if 'J' in str(seq[key]):
                break
            else:
                # store the sequence
                s_seq[key] = str(seq[key])
                
                sequences_class.append(mapping[file.split('/')[2]])

        sequences_dict[file.split('/')[2]] = s_seq

    return sequences_class, sequences_dict

mapping = {file.split('/')[2]: i for i, file in enumerate(files)}

# sequences_class, sequences_dict = get_seq_class(files, mapping)
    
# with open('./esm2_embedding/esm_embedding_label.pkl', 'wb') as f:
#     pickle.dump(sequences_class, f)

# with open('./esm2_embedding/esm_sequences_dict.pkl', 'wb') as f:
#     pickle.dump(sequences_dict, f)


with open('./esm2_embedding/esm_embedding_label.pkl', 'rb') as f:
    sequences_class = pickle.load(f)

with open('./esm2_embedding/esm_sequences_dict.pkl', 'rb') as f:
    sequences_dict = pickle.load(f)
    

In [2]:
import pickle
import glob
from tqdm import tqdm
import torch.nn as nn
import torch

In [5]:

# file_base = './esm2_embedding/esm_embedding_0_3.pkl'

# with open('./esm2_embedding/esm_embedding_0_3.pkl', 'rb') as f:
#     d = pickle.load(f)

files = ['./esm2_embedding/esm_embedding_0_3.pkl',
        './esm2_embedding/esm_embedding_4_7.pkl',
            './esm2_embedding/esm_embedding_8_10.pkl',
            './esm2_embedding/esm_embedding_11_14.pkl']



X_embedding_dataset_AVG = []
X_embedding_dataset_MAX = []
y_embedding_dataset = []
label = 0

adaptiveAvgPool = nn.AdaptiveAvgPool2d((512, 512))
for file in files:  
    with open(file, 'rb') as f:
        d = pickle.load(f)

    for i in range(len(d)):
        counter = 0
        print(i)

        for j in range(len(d[i])):
            if counter == 350:
                label = label + 1
                break

            d[i][j] = adaptiveAvgPool(d[i][j].type(torch.float32)).type(torch.float16)

            for k in range(len(d[i][j])):
                if counter == 350:
                    break
                X_embedding_dataset_AVG.append(d[i][j][k])
                y_embedding_dataset.append(label)
                counter += 1
    
    del d
    # print(len(d))
    # print(len(d[0]))
    # print([i[0].shape for i in d[0]])
    # print([i[0].shape for i in d[1]])
    # print([i[0].shape for i in d[2]])
    # try:
    #     print([i.shape for i in d[3]])
    # except:
    #     pass


# embedding_dataset


0
1
2
3
0
1
2
3
0
1
2
0
1
2
3


In [6]:
# get number of megabytes
import sys
# print(sys.getsizeof(X_embedding_dataset))

# # load the sequences and the target class
# with open('./esm2_embedding/esm_embedding_label.pkl', 'rb') as f:
#     sequences_class = pickle.load(f)

# with open('./esm2_embedding/esm_sequences_dict.pkl', 'rb') as f:
#     sequences_dict = pickle.load(f)


# with open('./esm2_embedding/esm_X_embedding_dataset_AVG.pkl', 'wb') as f:
#     pickle.dump(X_embedding_dataset_AVG, f)

# with open('./esm2_embedding/esm_embedding_y_dataset.pkl', 'wb') as f:
#     pickle.dump(y_embedding_dataset, f)


torch.save(torch.stack(X_embedding_dataset_AVG), './esm2_embedding/esm_X_embedding_dataset_AVG.pt')
torch.save(torch.tensor(y_embedding_dataset), './esm2_embedding/esm_embedding_y_dataset.pt')


In [13]:



# with open('./esm2_embedding/esm_X_embedding_dataset_AVG.pkl', 'rb') as f:
#     X_embedding_dataset_AVG = pickle.load(f)

# with open('./esm2_embedding/esm_embedding_y_dataset.pkl', 'rb') as f:
#     y_embedding_dataset = pickle.load(f)

# save



# # number of samples
# print(len(X_embedding_dataset_AVG))
# print(len(y_embedding_dataset))


# from sklearn.model_selection import train_test_split
# import numpy as np

# # split the dataset
# X_train, X_test, y_train, y_test = train_test_split(X_embedding_dataset_AVG, y_embedding_dataset, test_size=0.2, random_state=42)

# # cast the dataset as numpy array
# X_train = np.array(X_train)
# X_test = np.array(X_test)

# # cast the dataset as torch tensor
# X_train = torch.from_numpy(X_train)
# X_test = torch.from_numpy(X_test)

# # cast the dataset as torch tensor
# y_train = torch.tensor(y_train)
# y_test = torch.tensor(y_test)

# # print(X_train.shape)
# # print(X_test.shape)

# # print(y_train.shape)
# # print(y_test.shape)

# from torch.utils.data import TensorDataset, DataLoader

# # create the dataset
# train_dataset = TensorDataset(X_train, y_train)

# # create the dataset
# test_dataset = TensorDataset(X_test, y_test)

# # create the dataloader
# train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# # create the dataloader
# test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=True)

# # print(len(train_dataloader))
# # print(len(test_dataloader))

# # for i, (X, y) in enumerate(train_dataloader):
# #     print(X.shape)
# #     print(y.shape)
# #     break



torch.Size([3750, 256, 256])